In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name = 'tsbacktest')

Found the config file in: C:\Users\honglu\OneDrive - Microsoft\Projects\ForecastBenchmark\code\TSPerf\energy_load\GEFCom2017_D_Prob_MT_hourly\submissions\fnn\config.json


In [2]:
from azureml.core.runconfig import RunConfiguration
run_config_user_managed = RunConfiguration()
run_config_user_managed.environment.python.user_managed_dependencies = True
run_config_user_managed.environment.python.interpreter_path = 'C:/Users/honglu/AppData/Local/conda/conda/envs/amlsdk/python.exe'

In [3]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='./', 
                      script='aml_estimator.py', 
                      arguments=['--n_hidden_1', '5',
                                 '--n_hidden_2', '5',
                                 '--iter_max', '3',
                                 '--penalty', '0',
                                 '--path', './data/'],
                      run_config=run_config_user_managed)
run_local = exp.submit(src)

In [23]:
run_local.get_details()

{'runId': 'tsbacktest_1547507062_ee88896c',
 'target': 'local',
 'status': 'Failed',
 'startTimeUtc': '2019-01-14T23:04:26.173053Z',
 'endTimeUtc': '2019-01-14T23:04:36.293174Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'e4d4f433-e3d3-4593-bc25-9f33e06b3cc8'},
 'runDefinition': {'Script': 'aml_estimator.py',
  'Arguments': ['--n_hidden_1',
   '5',
   '--n_hidden_2',
   '5',
   '--iter_max',
   '3',
   '--penalty',
   '0',
   '--path',
   './data/'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'C:/Users/honglu/AppData/Local/conda/conda/envs/amlsdk/python.exe',
    'UserManagedDependencies': True,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2', {'pip': ['azureml-defaults']}]},
    'Con

In [6]:
run_local.get_metrics()

{'average pinball loss': 1.0}

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# choose a name for your cluster
compute_name =  "hlutsperfnn"
compute_min_nodes = 0
compute_max_nodes = 16

vm_size = "STANDARD_D3_V2"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

found compute target. just use it. hlutsperfnn


In [4]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas', 'r-base', 'r-data.table', 'r-rjson', 'r-optparse', 'r-doparallel'], python_version='3.6.2')
env.python.conda_dependencies.add_channel('conda-forge')
env.docker.enabled=True

In [7]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator

script_folder = './'

script_params = {
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0,
    '--path': ws.get_default_datastore().as_mount(),
    '--cv_path': ws.get_default_datastore().as_mount()
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                environment_definition=env)

In [8]:
run_batchai = exp.submit(config=est)

In [9]:
run_batchai.get_details()

{'runId': 'tsbacktest_1547580754791',
 'target': 'hlutsperfnn',
 'status': 'Starting',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'd8f82d4c-dac0-41a9-9d54-f07dc5ebce23'},
 'runDefinition': {'Script': 'aml_estimator.py',
  'Arguments': ['--n_hidden_1',
   '5',
   '--n_hidden_2',
   '5',
   '--iter_max',
   '3',
   '--penalty',
   '0',
   '--path',
   '$AZUREML_DATAREFERENCE_workspacefilestore',
   '--cv_path',
   '$AZUREML_DATAREFERENCE_workspacefilestore'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'hlutsperfnn',
  'DataReferences': {'workspacefilestore': {'DataStoreName': 'workspacefilestore',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependenc

In [12]:
from azureml.train.hyperdrive import *

script_folder = './'

script_params = {
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0,
    '--path': ws.get_default_datastore().as_mount(),
    '--cv_path': ws.get_default_datastore().as_mount()
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                environment_definition=env)

# ps = RandomParameterSampling({
#     '--n_hidden_1': choice(5, 10), 
#     '--n_hidden_2': choice(5, 10),
#     '--iter_max': choice(2, 4, 6, 8, 10),
#     '--penalty': choice(0),
# })

ps = GridParameterSampling({
        '--n_hidden_1': choice(8, 16), 
        '--n_hidden_2': choice(4, 8),
        '--iter_max': choice(10, 50, 100),
        '--penalty': choice(0, 0.1, 0.001),
    
})

htc = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          primary_metric_name='average pinball loss', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_concurrent_runs=16,
                          max_total_runs=40)
htr = exp.submit(config=htc)

The same input parameter(s) are specified in estimator script params and HyperDrive parameter space. HyperDrive parameter space definition will override duplicate entries in estimator. ['--n_hidden_1', '--n_hidden_2', '--iter_max', '--penalty'] is the list of overridden parameter(s).
